In [ ]:
import sys
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
from sim_pipeline.gg_lens_pop import GGLensPop
import lsst.daf.butler as dafButler
from sim_pipeline import rsp
from sim_pipeline.Util import param_util

## Lens injection in multiple DC2 cutout images

In this notebook, we will show how one can inject simulated lenses to multiple DC2 cutout images and

create a catalog of strong lense. and create a catalog of strong lense.

For this, we should follow following steps:
1. Create a lens population using sim-pipeline
2. Generate random ra and dec using random_ra_dec function in RSP module
3. Then, finally one can should call multiple_lens_injection function in RSP module

In [ ]:
## Users should change this path to their sim-pipeline path
sys.path.insert(0, '/home/nkhadka/notebooks/mytutorials/sim-pipeline/')

## Generate Galaxy-Galaxy Lens Population

In [ ]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit='deg2')

# define limits in the intrinsic deflector and source population (in addition to the skypy config 
# file)
kwargs_deflector_cut = {'band': 'g', 'band_max':28, 'z_min': 0.01, 'z_max': 2.5}
kwargs_source_cut = {'band': 'g', 'band_max':28, 'z_min': 0.1, 'z_max': 5.}

# run skypy pipeline and make galaxy-galaxy population class using GGLensPop
gg_lens_pop = GGLensPop(lens_type='all-galaxies', source_type='galaxies', 
                        kwargs_deflector_cut=kwargs_deflector_cut, kwargs_source_cut=
                        kwargs_source_cut,kwargs_mass2light=None, skypy_config=None, 
                        sky_area=sky_area, cosmo=cosmo)

## Initiate Butler
Butler is necessary to access and process DC2 data. So, one needs to initiate it. To initiate butler,

one needs to specify data configuration and data collection.

In [ ]:
## Here we configure Rubin Observatory’s Data Preview 0.2 (dp02) . Data Preview 0 (dp0) is
## the first of three data previews during the period leading up to the start of Rubin 
## Observatory Operations. With the new release of dp0 data, one can use the new version
## in the future.
config = "dp02"
collection ='2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collection)
skymap = butler.get("skyMap")

## Import rsp module
This module can be used to do all the operation necessary for the lens injection in DC2 data.
We will choose 

100 random ra, dec pair in the DC2 data and we inject random gg lenses in these ra, dec position. 

Injected_lens_catalog contains following images and information,


1. Lens to be injected in r-band.
2. Corresponding DC2 cutout image in r-band.
3. r, g, and i band DC2 cutout images with corresponding injected lenes.
4. Center of the DC2 cutout images where lenses have been injected.

Rsp module is imported at the top

In [ ]:
##provide minimum and maximum limits for ra and dec. Also, specify how many ra, dec pair you want.
ra, dec=param_util.random_ra_dec(55, 70, -43, -30, 100)

In [ ]:
## specify pixel number and pixel scale of the images. Here we have chosen 64 and 0.2 respectively. 
## We should choose pixel scale = 0.2 to match the pixel scale scale of DC2 data.
## For this function one lens_cut. If not provided, default option is None.
lens_cut = {'min_image_separation': 0.8, 'max_image_separation': 10, 
                        'mag_arc_limit': {'g': 23, 'r': 23, 'i': 23}}
injected_lens_catalog=rsp.multiple_lens_injection(gg_lens_pop, 64, 0.2, ra, dec, butler, 
lens_cut=lens_cut, flux=None)

In [ ]:
## This line should display an astropy table containg lens image,dp0 cutout_image, injected_lens 
## in r, g, and i band and  center of the dp0 cutout images.
injected_lens_catalog

## Fast injection
The function multiple_lens_injection() is capable of injecting multiple lenses to the multiple dp0 cutout images. This function injects one lens per tract/patch ID. So, for each lens, it needs to query dp0 data. The time taken by the lens injection process is mostly contributed by the dp0 data query. Therefore, it takes significantly large time to inject a given number of lenses to the dp0 cutouts. So, one can reduce the lens injection time by drawing multiple cutout images from the same tract/patch ID. This can be done by using the function multiple_lens_injection_fast() (see documentation for more detailed explanation).

In [ ]:
## This line should display an astropy table containg lens image,dp0 cutout_image, injected_lens 
## in r, g, and i band and  center of the dp0 cutout images
kwargs_lens_cut={'min_image_separation': 0.8, 'max_image_separation': 10, 'mag_arc_limit': {'g': 23, 'r': 23, 'i': 23}}
ra_2, dec_2=param_util.random_ra_dec(55, 70, -43, -30, 10)
injected_lens_catalog_2 = rsp.multiple_lens_injection_fast(gg_lens_pop, 100, 0.2, butler, ra, dec, 10, kwargs_lens_cut, flux=None)
injected_lens_catalog_2